In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [50]:
def regress(X, y, regr, index):
	train = X[index].to_numpy()
	print(train)
	regr.fit(train, y)
	for i in train:
		i.reshape(-1, 1)
	y_pred = regr.predict(X)
	print("Mean squared error on train: ", mean_squared_error(y, y_pred))
	print("Training R2: ", r2_score(y, y_pred))



In [54]:
X = pd.read_csv("TWTR.csv")
X = X[['Open', 'High', 'Low', 'Close', 'Volume']]
y = np.zeros((X.shape[0] // 10))
X = np.array_split(X, X.shape[0] / 10)

for i in range(len(X)):
	stock_open = X[i].Open.to_numpy()
	stock_close = X[i].Close.to_numpy()
	y[i] = round(stock_close[9] - stock_open[5], 2)

print(len(X), len(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(len(X_train), len(y_train))

for i in range(len(X_train)):
	X_train[i] = X_train[i][:5]

for i in range(len(X_test)):
	X_test[i] = X_test[i][:5]


223 223
178 178


In [52]:
regr = linear_model.LinearRegression()
regress(X_train, y_train, regr, 0)


[[3.1700001e+01 3.2049999e+01 3.0910000e+01 3.1799999e+01 1.8915169e+07]
 [3.1799999e+01 3.2400002e+01 3.1540001e+01 3.2250000e+01 1.4554404e+07]
 [3.3080002e+01 3.3500000e+01 3.2259998e+01 3.3090000e+01 3.0504455e+07]
 [3.2849998e+01 3.3200001e+01 3.2430000e+01 3.2869999e+01 1.7732289e+07]
 [3.2380001e+01 3.2750000e+01 3.2119999e+01 3.2369999e+01 9.5230300e+06]]


ValueError: Found input variables with inconsistent numbers of samples: [5, 178]